In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 


X = heart_disease.data.features 
y = heart_disease.data.targets 
  
hd_df = pd.DataFrame(data = X) 
hd_df['Outcome'] = y
hd_df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,Outcome
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


# Question 1

A Neural Network is an AI model that is inspired and modeled after the way the human brain operates. It's used for solving complex problems in various areas like pattern recognition, classification, regression, and more.
the general steps to building a nueral network are:
Data Collection and Preprocessing,Define Network Architecture,
Initialize the Model,Forward Propagation,Compute Loss,Backpropagation,
Iterate Training,Validation and Tuning, Testing,Deployment and Monitoring



# Question 2

Generally we check the performance of a nueral network using metrics like accuracy, loss, f1 score. these metrics provide insights into how well the model is learning and generalizing from the data. 

# Question 3

In [52]:
hd_df= hd_df[hd_df['Outcome'] <= 1]


In [53]:
hd_df = hd_df.dropna()

In [54]:
hd_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,Outcome
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
5,56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


this data didnt really need more cleaning. 

# Question 4

In [83]:
X = hd_df.drop('Outcome', axis=1).values
y = hd_df['Outcome'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=32,
                                                    stratify=y)


In [103]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_test = torch.LongTensor(y_test)
y_train = torch.LongTensor(y_train)

print(X_train)

tensor([[59.,  0.,  4.,  ...,  2.,  0.,  3.],
        [50.,  0.,  3.,  ...,  2.,  0.,  3.],
        [52.,  0.,  3.,  ...,  2.,  0.,  3.],
        ...,
        [42.,  1.,  4.,  ...,  1.,  0.,  3.],
        [67.,  0.,  3.,  ...,  2.,  0.,  7.],
        [57.,  1.,  2.,  ...,  1.,  1.,  3.]])


In [203]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=13,hidden1=20,hidden2=20,hidden3=20, out_features=2):
        super().__init__() 
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.layer_3_connection = nn.Linear(hidden2, hidden3)
        self.out = nn.Linear(hidden3, out_features)

    def forward(self, x):
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = F.relu(self.layer_3_connection(x))
        x = self.out(x)
        return x

In [204]:
torch.manual_seed(42)
Ann = ANN_Model()

In [205]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.RAdam(Ann.parameters(), lr=0.01)

In [206]:
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = Ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)

    if epoch % 20 ==1:
        print(f'Epoch number: {epoch} with loss: {loss}')

    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 

Epoch number: 1 with loss: 0.6672568321228027
Epoch number: 21 with loss: 0.5448777079582214
Epoch number: 41 with loss: 0.5242448449134827
Epoch number: 61 with loss: 0.506575345993042
Epoch number: 81 with loss: 0.4858362674713135
Epoch number: 101 with loss: 0.46851834654808044
Epoch number: 121 with loss: 0.45053961873054504
Epoch number: 141 with loss: 0.4245365858078003
Epoch number: 161 with loss: 0.3899588882923126
Epoch number: 181 with loss: 0.3628610074520111
Epoch number: 201 with loss: 0.35431939363479614
Epoch number: 221 with loss: 0.34160828590393066
Epoch number: 241 with loss: 0.33725205063819885
Epoch number: 261 with loss: 0.33825889229774475
Epoch number: 281 with loss: 0.32957035303115845
Epoch number: 301 with loss: 0.33343443274497986
Epoch number: 321 with loss: 0.3269016742706299
Epoch number: 341 with loss: 0.3250870704650879
Epoch number: 361 with loss: 0.3225403428077698
Epoch number: 381 with loss: 0.32187095284461975
Epoch number: 401 with loss: 0.3202621

In [207]:
y_pred = []

with torch.no_grad(): 
    for i, data in enumerate(X_test):
        prediction = Ann(data)
        y_pred.append(prediction.argmax()) 

In [208]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81        49
           1       0.38      0.31      0.34        16

    accuracy                           0.71        65
   macro avg       0.59      0.57      0.58        65
weighted avg       0.69      0.71      0.70        65



# Question 5

the neural network performed the best with the RAdam optimizer. However it still performed worse than other models like 
Decision Trees and XGB boost. 